# 네이버 영화 140자평 리뷰 자료 스크레이핑 프로토타입


네이버 영화 웹 서비스의 평점과 140자평 데이터를 스크레이핑하여 데이터 셋으로 만드는 예제입니다.

- 네티즌 평점 / 140자평 : https://movie.naver.com/movie/point/af/list.nhn?target=after


step 1. 평점 개수 확인

>#old_content > h5 > div > strong

step 2. 데이터셋 생성

step 3. 데이터 저장

가끔 한글이 깨지는 경우가 있습니다. 아래 두줄의 코드를 넣어줍니다.

In [1]:
#! /usr/bin/python3
# -*- coding: utf-8 -*-

**라이브러리 import**

In [2]:
import numpy as np
import pandas as pd
 
import urllib.request as req
import re # 정규식 관련 라이브러리
from bs4 import BeautifulSoup # html 파싱용 라이브러리

In [3]:
BASE_URL = "https://movie.naver.com/movie/point/af/list.nhn?target=after"
PAGE_URL = BASE_URL + "?&page=%s"

**리뷰 페이지가 몇개인지 먼저 확인합니다.**

In [4]:
html = req.urlopen(BASE_URL)
soup = BeautifulSoup(html, "html.parser")

review_count = soup.select("#old_content > h5 > div > strong")
review_count = int(review_count[0].string)

review_count

11271869

In [5]:
page_count = int(np.ceil(review_count / 10)) # 한 페이지당 10개의 게시글
page_count

1127187

## 페이지 리턴 함수 정의

In [6]:
def count_page(url):
    html = req.urlopen(url)
    soup = BeautifulSoup(html, "html.parser")
    
    review_count = soup.select("#old_content > h5 > div > strong")
    review_count = int(review_count[0].string)
    
    return int(np.ceil(review_count / 10)) # 한 페이지당 10개의 게시글

In [7]:
count_page(BASE_URL)

1127187

**테스트용 1페이지 파싱**

In [8]:
page_num = 1
URL = PAGE_URL % page_num
URL

'https://movie.naver.com/movie/point/af/list.nhn?target=after?&page=1'

In [9]:
res = req.urlopen(URL)
soup = BeautifulSoup(res, "html.parser")

In [10]:
test_tr = soup.find_all('tr')

**첫번째 행은 Header 정보입니다.**

In [11]:
test_tr[0]

<tr>
<th>번호</th>
<th colspan="2">평점</th>
<th>140자평</th>
<th class="al"><span class="th_m1">글쓴이·날짜</span></th>
</tr>

## 영화 리뷰에 대한 정보입니다. 여기서 필요한 것만 파싱합니다.

1. 리뷰 번호
2. 리뷰 평점
3. 영화 제목
4. 영화 리뷰

In [12]:
test_tr[1]

<tr>
<td class="ac num">14770345</td>
<td><div class="fr point_type_n"><div class="mask" style="width:50.0%"></div></div></td>
<td class="point">5</td>
<td class="title">
<a class="movie" href="?st=mcode&amp;sword=162981&amp;target=after">명당</a>
<br/>평이하고 진부하여 감흥 조차 둔감해지는. 
			
			
			
				
				
				
				<a class="report" href="javascript:report('gato****', 'joFc16KDDyXYfVt6H54s2v2fldFZoCO9KFH2DA0r+eM=', '평이하고 진부하여 감흥 조차 둔감해지는.', '14770345', 'point_after');" style="color:#8F8F8F">신고</a>
</td>
<td class="num"><a class="author" href="javascript:find_list('nickname','14770345','after');">gato****</a><br/>18.10.24</td>
</tr>

### 1. 리뷰 번호

In [13]:
test_num = test_tr[1].find('td', {'class':'ac num'})
test_num = test_num.text
test_num

'14770345'

### 2. 리뷰 평점

In [14]:
test_point = test_tr[1].find('td', {'class':'point'})
test_point = test_point.text
test_point

'5'

**영화의 대한 정보**

In [15]:
test_movie_info = test_tr[1].find('td', {'class':'title'})
test_movie_info = test_movie_info.text
test_movie_info

'\n명당\n평이하고 진부하여 감흥 조차 둔감해지는. \r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\t\r\n\t\t\t\t\r\n\t\t\t\t\r\n\t\t\t\t신고\n'

In [16]:
movielist = test_movie_info.split('\n')

### 3. 영화 제목

In [17]:
test_title = movielist[1].strip()
test_title

'명당'

### 4. 영화 리뷰

In [18]:
test_review = movielist[2].strip()
test_review

'평이하고 진부하여 감흥 조차 둔감해지는.'

## 파싱 함수 만들기

위의 내용을 토대로 파싱 함수를 만듭니다.

In [19]:
def parse_item(item):
    # return 리뷰번호, 평점, 영화제목, 영화리뷰
    # num, point, title, review
    
    _num = item.find('td', {'class':'ac num'})
    _num = _num.text

    _point = item.find('td', {'class':'point'})
    _point = _point.text
    
    movie_info = item.find('td', {'class':'title'})
    movie_info = movie_info.text
    movie_info = movie_info.split('\n')
    
    _title = movie_info[1].strip()
    _review = movie_info[2].strip()
          
    return {'num': _num, 'point': _point, 'title': _title, 'review': _review}

In [20]:
def scraping_rating_page(url):   
    html = req.urlopen(url)
    soup = BeautifulSoup(html, 'html')
    
    items = soup.findAll('tr')
    items = items[1:] # 첫번째 행은 header 이므로 제거합니다.

    return list(filter(None, [parse_item(item) for item in items]))

In [21]:
test_list = scraping_rating_page(URL)

In [22]:
test_list

[{'num': '14770347',
  'point': '10',
  'title': '트로이',
  'review': '아킬레스 사후 아들인 네오프톨레모스가 참전해서 트로이 멸망. 헥토르의 아내 안드로마케를 노예로 끌고가 자식을 낳음. 네오프톨레모스 사후 안드로마케는 도련님인 헬레노스와 결혼해 새 왕국 건설. 아프로디테 아들 아이네이아스가 트로이에서 도망쳐 로마 건설.'},
 {'num': '14770345',
  'point': '5',
  'title': '명당',
  'review': '평이하고 진부하여 감흥 조차 둔감해지는.'},
 {'num': '14770346',
  'point': '10',
  'title': '퍼스트맨',
  'review': '건조한 연출, 잔잔한 울림. 감동적으로 보았다. 가타카나 클린트 이스트 우드 류의 건조한 영화 좋아하는 사람들에겐 최고의 영화일테고, 다 때려부수고 욕하고 죽이고 이런 킬링타임 혹은 블록버스터 영화 좋아하는 사람들한테는 안맞을거임.'},
 {'num': '14770344',
  'point': '10',
  'title': '곰돌이 푸 다시 만나 행복해',
  'review': '가족영화의 정석.. ㅜㅜ'},
 {'num': '14770343',
  'point': '10',
  'title': '펭귄 하이웨이',
  'review': '더빙판 봤는데 재미있게 잘 봤네요^^ 아이들이 보기엔 내용이 다소 난해하지만, 그래도 작화도 아름답고 상상력을 자극하는 내용이라 추천합니다!'},
 {'num': '14770342',
  'point': '10',
  'title': '스타 이즈 본',
  'review': '음악도 진짜 좋았지만 이건 분명 사랑을 다룬 영화다.'},
 {'num': '14770341',
  'point': '10',
  'title': '스타 이즈 본',
  'review': '노래가 너무 좋아서 귀가 호강햇습니다..'},
 {'num': '14770340',
  'point': '

**10개의 리뷰 정보가 저장되었습니다. (1페이지 분량)**

In [23]:
df = pd.DataFrame(columns=('num', 'point', 'title', 'review'))
df.append(test_list)

,num,point,title,review
0,14770347,10,트로이,아킬레스 사후 아들인 네오프톨레모스가 참전해서 트로이 멸망. 헥토르의 아내 안드로마...
1,14770345,5,명당,평이하고 진부하여 감흥 조차 둔감해지는.
2,14770346,10,퍼스트맨,"건조한 연출, 잔잔한 울림. 감동적으로 보았다. 가타카나 클린트 이스트 우드 류의 ..."
3,14770344,10,곰돌이 푸 다시 만나 행복해,가족영화의 정석.. ㅜㅜ
4,14770343,10,펭귄 하이웨이,"더빙판 봤는데 재미있게 잘 봤네요^^ 아이들이 보기엔 내용이 다소 난해하지만, 그래..."
5,14770342,10,스타 이즈 본,음악도 진짜 좋았지만 이건 분명 사랑을 다룬 영화다.
6,14770341,10,스타 이즈 본,노래가 너무 좋아서 귀가 호강햇습니다..
7,14770340,8,천하장사 마돈나,이해영감독을 기억하게 하는 유일한 영화...뻔하지 않아서 좋았다
8,14770339,8,암수살인,재밋게봣어요 범인녀석 반성해야됨
9,14770338,10,미쓰백,그냥 말이 필요없다. 그간 봐왔던 한지민 팬으로써 연기력에 감탄하고 부단한 노력을 ...


**테스트로 100개의 데이터를 읽어와서 csv 로 저장해보겠습니다.**

In [24]:
test_df = pd.DataFrame(columns=('num', 'point', 'title', 'review'))

for index in range(10):
    test_df = test_df.append(scraping_rating_page(PAGE_URL % (index+1)), ignore_index=True)
    
test_df

,num,point,title,review
0,14770349,10,프리퀀시,"프랭크 설리반 같은 아빠, 줄리아 설리반 같은 아내, 존 설리반 같은 아들, 고르도..."
1,14770348,10,너의 이름은.,와...밑에 내용이해안된다고 벌점테러한 인간이있네지가 빡대가리인것도 모르고ㅋㅋㅋ
2,14770347,10,트로이,아킬레스 사후 아들인 네오프톨레모스가 참전해서 트로이 멸망. 헥토르의 아내 안드로마...
3,14770345,5,명당,평이하고 진부하여 감흥 조차 둔감해지는.
4,14770346,10,퍼스트맨,"건조한 연출, 잔잔한 울림. 감동적으로 보았다. 가타카나 클린트 이스트 우드 류의 ..."
5,14770344,10,곰돌이 푸 다시 만나 행복해,가족영화의 정석.. ㅜㅜ
6,14770343,10,펭귄 하이웨이,"더빙판 봤는데 재미있게 잘 봤네요^^ 아이들이 보기엔 내용이 다소 난해하지만, 그래..."
7,14770342,10,스타 이즈 본,음악도 진짜 좋았지만 이건 분명 사랑을 다룬 영화다.
8,14770341,10,스타 이즈 본,노래가 너무 좋아서 귀가 호강햇습니다..
9,14770340,8,천하장사 마돈나,이해영감독을 기억하게 하는 유일한 영화...뻔하지 않아서 좋았다


**리뷰 갯수가 딱 맞지 않는 이유는 스크레핑 하는 과정에도 새로운 게시물이 생겨서 페이징 시에 누락되는 평점이 존재 하기 때문입니다.**

##### csv 형태로 저장

내용이 많으므로 50 페이지 당 한번씩 Raw 데이터를 csv 파일로 저장합니다.

In [ ]:
review_df = pd.DataFrame(columns=('num', 'point', 'title', 'review'))
review_file = '../data/raw/%s.csv'

for index in range(count_page(BASE_URL)):
    review_df.
    
    review_df = review_df.append(scraping_rating_page(PAGE_URL % (index+1)), ignore_index=True)
    
    if ((index+1) % 100) == 0:
        file_name = review_file % (index+1)
        review_df.to_csv(file_name, index=False, header=True)
        
        review_df = pd.DataFrame(columns=('num', 'point', 'title', 'review')) # 저장후 초기화
        
        print(file_name, '저장했습니다.')